# 1.1 🚀 Initialization and Configuration

* In this section, necessary libraries are imported, and the Semantic Kernel is configured with Azure OpenAI credentials loaded from a configuration file.

In [84]:
#!import config/Settings.cs

In [85]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta8"

Installed Packages Microsoft.SemanticKernel, 1.0.0-beta8

In [86]:
using Microsoft.SemanticKernel;

var builder = new KernelBuilder();

In [87]:
using Kernel = Microsoft.SemanticKernel.Kernel;

var (model1, azureEndpoint1, apiKey1, gpt35TurboServiceId) = Settings.LoadFromFile(model: "gpt-35-turbo");
builder.WithAzureOpenAIChatCompletionService(model1, azureEndpoint1, apiKey1, serviceId: gpt35TurboServiceId);

var (model2, azureEndpoint2, apiKey2, gpt4ServiceId) = Settings.LoadFromFile(model: "gpt-4-32k");
builder.WithAzureOpenAIChatCompletionService(model2, azureEndpoint2, apiKey2, serviceId: gpt4ServiceId);

var kernel = builder.Build();

# 1.2 🚀 Semantic Function for Short Intent Extraction
* This section focuses on setting up the Semantic Function for extracting a concise user intent using a specific prompt.

In [88]:
using Microsoft.SemanticKernel.TemplateEngine;
using Microsoft.SemanticKernel.TemplateEngine.Basic;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
using Microsoft.SemanticKernel.AI;

var aiRequestSettings = new AIRequestSettings 
{
    ExtensionData = new Dictionary<string, object> { { "api-version", "2023-03-15-preview" } },
    ServiceId = gpt35TurboServiceId
};

In [89]:
var input = "I want to find top-10 books about world history";

In [90]:
// Chat with the Bot
// Create a simple prompt for the semantic function

string skPrompt = @"ChatBot: How can I help you?
User: {{$input}}

---------------------------------------------

The intent of the user in 25 words or less: ";

var getShortIntentFunction  = kernel.CreateSemanticFunction(skPrompt, "fn", "pn", requestSettings: aiRequestSettings);

var intentResult = await kernel.RunAsync(input, getShortIntentFunction);

Console.WriteLine(intentResult);

The user wants recommendations for the top 10 books about world history.


# 1.3 🚀 Semantic Function for Conversational Chat

* This section defines the Semantic Function for engaging in a more extended conversation with the chatbot, initializing the context, and executing the function.

In [91]:
skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give detailed answer or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

In [92]:
// Chat with the Bot

var aiRequestSettings = new AIRequestSettings 
{
    ExtensionData = new Dictionary<string, object> { { "api-version", "2023-03-15-preview" } },
    ServiceId = gpt4ServiceId
};

var chatFunction = kernel.CreateSemanticFunction(skPrompt, "fn", "pn", requestSettings: aiRequestSettings);
var context = kernel.CreateNewContext();

var history = "";
context.Variables["history"] = history;

var userInput = "Hi, I'm looking for suggestions of best sellers about world history";
context.Variables["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(context);

In [93]:
// Update the history with the output and set this as the new input value for the next request

history += $"\nUser: {userInput}\nSuggestions: {bot_answer.GetValue<string>()}\n";
context.Variables.Update(history);

Console.WriteLine(context);


User: Hi, I'm looking for suggestions of best sellers about world history
Suggestions: Sure, here are some best-selling books about world history that you might find interesting:

1. "Sapiens: A Brief History of Humankind" by Yuval Noah Harari
2. "Guns, Germs, and Steel: The Fates of Human Societies" by Jared Diamond
3. "A People's History of the United States" by Howard Zinn
4. "The Silk Roads: A New History of the World" by Peter Frankopan
5. "The History of the World in Six Glasses" by Tom Standage
6. "The World: A Brief Introduction" by Richard Haass
7. "The Penguin History of the World" by J.M. Roberts
8. "The Story of Civilization" by Will Durant
9. "The Lessons of History" by Will and Ariel Durant
10. "The Rise and Fall of the Great Powers" by Paul Kennedy

Remember to check the availability of these books in your local library or bookstore. Happy reading!



# 1.4 🚀 Interactive Chat Function

* In this final section, an interactive chat function is defined, and several interactions with the chatbot are simulated. The complete chat history and the final user input are then printed. *

In [94]:
// Chatting more)

Func<string, Task> Chat = async (string input) => {
    // Save new message in the context variables
    context.Variables["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(context);

    // Append the new interaction to the chat history
    history += $"\nUser: {input}\nSuggestions: {answer.GetValue<string>()}\n"; 
    context.Variables["history"] = history;
    
    // Show the response
    Console.WriteLine(context);
};

In [95]:
await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");

Sure, I would recommend "The History of the Peloponnesian War" by Thucydides. It's a classic work of ancient Greek history that provides a detailed account of the war between Athens and Sparta in the 5th century BC.


In [96]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");

"The History of the Peloponnesian War" by Thucydides covers a wide range of topics related to the war between Athens and Sparta, which was a significant conflict in ancient Greece. Here are some of the topics you can expect to learn about:

1. Political and military strategies: Thucydides provides detailed accounts of the strategies used by both sides in the war. He discusses the strengths and weaknesses of different approaches, and the impact of leadership decisions on the course of the war.

2. Diplomacy and alliances: The book explores the complex network of alliances that characterized the Greek city-states during this period. It provides insight into the diplomatic maneuvers and negotiations that took place.

3. Social and economic conditions: Thucydides also discusses the social and economic conditions in Athens and Sparta, and how these influenced the war. This includes the impact of the plague in Athens and the role of economic resources in the conflict.

4. Human nature and et

In [97]:
await Chat("Which topic from the ones you listed do you think the most popular?");

As an AI, I don't have personal opinions or the ability to gauge popularity. However, based on general interest, many readers find the political and military strategies discussed in "The History of the Peloponnesian War" particularly fascinating. This is because they provide a detailed look at the tactics and decisions that shaped a major conflict in ancient history. Additionally, Thucydides' reflections on human nature and ethics are often noted for their depth and relevance, even in the modern world.


In [98]:
Console.WriteLine(context.Variables["history"]);


User: Hi, I'm looking for suggestions of best sellers about world history
Suggestions: Sure, here are some best-selling books about world history that you might find interesting:

1. "Sapiens: A Brief History of Humankind" by Yuval Noah Harari
2. "Guns, Germs, and Steel: The Fates of Human Societies" by Jared Diamond
3. "A People's History of the United States" by Howard Zinn
4. "The Silk Roads: A New History of the World" by Peter Frankopan
5. "The History of the World in Six Glasses" by Tom Standage
6. "The World: A Brief Introduction" by Richard Haass
7. "The Penguin History of the World" by J.M. Roberts
8. "The Story of Civilization" by Will Durant
9. "The Lessons of History" by Will and Ariel Durant
10. "The Rise and Fall of the Great Powers" by Paul Kennedy

Remember to check the availability of these books in your local library or bookstore. Happy reading!

User: I would like a non-fiction book suggestion about Greece history. Please only list one book.
Suggestions: Sure, I wou

In [99]:
Console.WriteLine(context.Variables["userInput"]);

Which topic from the ones you listed do you think the most popular?
